In [1]:
import pickle
import argparse
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import dgl

import os 
import sys 
import pickle 
import pandas as pd
import numpy as np 
from pandas.api.types import is_numeric_dtype, is_categorical_dtype, is_categorical
import torch
import pandas as pd 
import dgl
import dgl.function as fn
from sklearn.model_selection import train_test_split

import numpy as np
import dgl
import torch
from torch.utils.data import IterableDataset, DataLoader


from torch import nn
from sklearn.metrics import roc_auc_score, ndcg_score
import tqdm 

# import layers
# import sampler as sampler_module
# import evaluation

Using backend: pytorch


## Sampler

In [2]:
class ItemToItemBatchSampler(IterableDataset):
    def __init__(self, g, user_type, item_type, batch_size):
        self.g = g
        self.user_type = user_type
        self.item_type = item_type
        self.user_to_item_etype = list(g.metagraph()[user_type][item_type])[0]
        self.item_to_user_etype = list(g.metagraph()[item_type][user_type])[0]
        self.batch_size = batch_size

    def __iter__(self):
        while True:
            heads = torch.randint(0, self.g.number_of_nodes(self.item_type), (self.batch_size,))
            tails = dgl.sampling.random_walk(
                self.g,
                heads,
                metapath=[self.item_to_user_etype, self.user_to_item_etype])[0][:, 2]
            neg_tails = torch.randint(0, self.g.number_of_nodes(self.item_type), (self.batch_size,))

            mask = (tails != -1)
            yield heads[mask], tails[mask], neg_tails[mask]

## Neighbourhood Sampler


In [43]:
def compact_and_copy(frontier, seeds):
    block = dgl.to_block(frontier, seeds)
    for col, data in frontier.edata.items():
        if col == dgl.EID:
            continue
        block.edata[col] = data[block.edata[dgl.EID]]
    return block

class NeighborSampler(object):
    def __init__(self, g, user_type, item_type, random_walk_length, random_walk_restart_prob,
                 num_random_walks, num_neighbors, num_layers):
        self.g = g
        self.user_type = user_type
        self.item_type = item_type
        self.user_to_item_etype = list(g.metagraph()[user_type][item_type])[0]
        self.item_to_user_etype = list(g.metagraph()[item_type][user_type])[0]
        self.samplers = [
            dgl.sampling.PinSAGESampler(g, item_type, user_type, random_walk_length,
                                        random_walk_restart_prob, num_random_walks, num_neighbors)
            for _ in range(num_layers)]

    def sample_blocks(self, seeds, heads=None, tails=None, neg_tails=None):
        blocks = []
        for sampler in self.samplers:
            frontier = sampler(seeds)
            if heads is not None:
                eids = frontier.edge_ids(torch.cat([heads, heads]), torch.cat([tails, neg_tails]), return_uv=True)[2]
                if len(eids) > 0:
                    old_frontier = frontier
                    frontier = dgl.remove_edges(old_frontier, eids)
                    # print(old_frontier)
                    # print(frontier)
                    # print(frontier.edata['weights'])
                    # frontier.edata['weights'] = old_frontier.edata['weights'][frontier.edata[dgl.EID]]
            block = compact_and_copy(frontier, seeds)
            seeds = block.srcdata[dgl.NID]
            blocks.insert(0, block)
        return blocks

    def sample_from_item_pairs(self, heads, tails, neg_tails):
        # Create a graph with positive connections only and another graph with negative
        # connections only.
        pos_graph = dgl.graph(
            (heads, tails),
            num_nodes=self.g.number_of_nodes(self.item_type))
        neg_graph = dgl.graph(
            (heads, neg_tails),
            num_nodes=self.g.number_of_nodes(self.item_type))
        pos_graph, neg_graph = dgl.compact_graphs([pos_graph, neg_graph])
        seeds = pos_graph.ndata[dgl.NID]

        blocks = self.sample_blocks(seeds, heads, tails, neg_tails)
        return pos_graph, neg_graph, blocks

def assign_features_to_blocks(blocks, g, ntype='track'):
    
    data = blocks[0].srcdata
    
    for col in g.nodes[ntype].data.keys():
        if  col == dgl.NID:
            continue
        induced_nodes = data[dgl.NID]
        data[col] = g.nodes[ntype].data[col][induced_nodes]

    
    data = blocks[-1].dstdata
    for col in g.nodes[ntype].data.keys():
        if  col == dgl.NID:
            continue
        induced_nodes = data[dgl.NID]
        data[col] = g.nodes[ntype].data[col][induced_nodes]

class PinSAGECollator(object):
    def __init__(self, sampler, g, ntype):
        self.sampler = sampler
        self.ntype = ntype
        self.g = g

    def collate_train(self, batches):
        heads, tails, neg_tails = batches[0]
        # Construct multilayer neighborhood via PinSAGE...
        pos_graph, neg_graph, blocks = self.sampler.sample_from_item_pairs(heads, tails, neg_tails)
        assign_features_to_blocks(blocks, self.g, self.ntype)

        return pos_graph, neg_graph, blocks
    
    def collate_test(self, samples):
        batch = torch.LongTensor(samples) 
        blocks = self.sampler.sample_blocks(batch)
        assign_features_to_blocks(blocks, self.g, self.ntype)
        return blocks

## Modeling

In [4]:
def disable_grad(module):
    for param in module.parameters():
        param.requires_grad = False


def _init_input_modules(g, ntype):
    module_dict = nn.ModuleDict()
    
    tracks_data = g.nodes[ntype].data
    
    module_dict['track_id'] = nn.Embedding(tracks_data['id'].max()+1, 128)
    
    for m in ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']: 
        module_dict[m] = nn.Embedding(tracks_data[m].max() + 1, 16)

    return module_dict

class LinearProjector(nn.Module):
    """
    Projects each input feature of the graph linearly and sums them up
    """

    def __init__(self, full_graph, ntype):
        super().__init__()

        self.ntype = ntype
        self.fc = nn.Linear(164, 128)
        #self.fc = nn.Linear(2212, 128)
        self.inputs = _init_input_modules(full_graph, ntype)

    def forward(self, ndata):
        
        # get music feature
        music_features = []
        for c in ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']:

            module = self.inputs[c]
            music_features.append(module(ndata[c]))
        music_features = torch.cat(music_features, dim=1)
        
        
#         # id embedding 
#         id_embedding = self.inputs['track_id'](ndata['id'])
        
#         # album feature 
#         img_emb = ndata['album_img_emb']
        
        # genre 
        genre = ndata['genre']
        
        # concatenate 
        #feature = torch.cat([music_features, genre, ndata['album_img_emb']], dim=1)
        feature = torch.cat([music_features, genre], dim=1)

        projection = self.fc(feature) #+ id_embedding
        
        return projection

## SageNet Layers

In [5]:
import torch.nn.functional as F


class WeightedSAGEConv(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims, act=F.relu):
        super().__init__()

        self.act = act
        self.Q = nn.Linear(input_dims, hidden_dims)
        self.W = nn.Linear(input_dims + hidden_dims, output_dims)
        self.reset_parameters()
        self.dropout = nn.Dropout(0.5)

    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_uniform_(self.Q.weight, gain=gain)
        nn.init.xavier_uniform_(self.W.weight, gain=gain)
        nn.init.constant_(self.Q.bias, 0)
        nn.init.constant_(self.W.bias, 0)

    def forward(self, g, h, weights):
        """
        g : graph
        h : node features
        weights : scalar edge weights
        """
        h_src, h_dst = h
        with g.local_scope():
            g.srcdata['n'] = self.act(self.Q(self.dropout(h_src)))
            g.edata['w'] = weights.float()
            g.update_all(fn.u_mul_e('n', 'w', 'm'), fn.sum('m', 'n'))
            g.update_all(fn.copy_e('w', 'm'), fn.sum('m', 'ws'))
            n = g.dstdata['n']
            ws = g.dstdata['ws'].unsqueeze(1).clamp(min=1)
            z = self.act(self.W(self.dropout(torch.cat([n / ws, h_dst], 1))))
            z_norm = z.norm(2, 1, keepdim=True)
            z_norm = torch.where(z_norm == 0, torch.tensor(1.).to(z_norm), z_norm)
            z = z / z_norm
            return z


class SAGENet(nn.Module):
    def __init__(self, hidden_dims, n_layers):
        """
        g : DGLHeteroGraph
            The user-item interaction graph.
            This is only for finding the range of categorical variables.
        item_textsets : torchtext.data.Dataset
            The textual features of each item node.
        """
        super().__init__()

        self.convs = nn.ModuleList()
        for _ in range(n_layers):
            self.convs.append(WeightedSAGEConv(hidden_dims, hidden_dims, hidden_dims))

    def forward(self, blocks, h):
        for layer, block in zip(self.convs, blocks):
            h_dst = h[:block.number_of_nodes('DST/' + block.ntypes[0])]
            h = layer(block, (h, h_dst), block.edata['weights'])
        return h

class ItemToItemScorer(nn.Module):
    def __init__(self, full_graph, ntype):
        super().__init__()

        n_nodes = full_graph.number_of_nodes(ntype)
        self.bias = nn.Parameter(torch.zeros(n_nodes))

    def _add_bias(self, edges):
        bias_src = self.bias[edges.src[dgl.NID]]
        bias_dst = self.bias[edges.dst[dgl.NID]]
        return {'s': edges.data['s'] + bias_src + bias_dst}

    def forward(self, item_item_graph, h):
        """
        item_item_graph : graph consists of edges connecting the pairs
        h : hidden state of every node
        """
        with item_item_graph.local_scope():
            item_item_graph.ndata['h'] = h
            item_item_graph.apply_edges(fn.u_dot_v('h', 'h', 's'))
            item_item_graph.edata['s'] = item_item_graph.edata['s'].flatten()
            item_item_graph.apply_edges(self._add_bias)

            pair_score = item_item_graph.edata['s']
        return pair_score


## Model

In [6]:
def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).detach().cpu().numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

class PinSAGEModel(nn.Module):
    def __init__(self, full_graph, ntype, hidden_dims, n_layers):
        super().__init__()

        self.proj = LinearProjector(full_graph, ntype)
        self.sage = SAGENet(hidden_dims, n_layers)
        self.scorer = ItemToItemScorer(full_graph, ntype)

    def forward(self, pos_graph, neg_graph, blocks):
        h_item = self.get_repr(blocks)
        pos_score = self.scorer(pos_graph, h_item)
        neg_score = self.scorer(neg_graph, h_item)
        
        #return h_item, pos_score, neg_score
        auc = compute_auc(pos_score, neg_score)
        return (neg_score - pos_score + 1).clamp(min=0), auc 


    def get_repr(self, blocks):
        h_item = self.proj(blocks[0].srcdata)
        h_item_dst = self.proj(blocks[-1].dstdata)
        return h_item_dst + self.sage(blocks, h_item)



## Evaluation

In [7]:
def prec(recommendations, ground_truth):
    n_users, n_items = ground_truth.shape
    K = recommendations.shape[1]
    user_idx = np.repeat(np.arange(n_users), K)
    item_idx = recommendations.flatten()
    relevance = ground_truth[user_idx, item_idx].reshape((n_users, K))
    hit = relevance.any(axis=1).mean()
    return hit

class LatestNNRecommender(object):
    def __init__(self, user_ntype, item_ntype, user_to_item_etype, timestamp, batch_size):
        self.user_ntype = user_ntype
        self.item_ntype = item_ntype
        self.user_to_item_etype = user_to_item_etype
        self.batch_size = batch_size
        self.timestamp = timestamp

    def recommend(self, full_graph, K, h_user, h_item):
        """
        Return a (n_user, K) matrix of recommended items for each user
        """
        graph_slice = full_graph.edge_type_subgraph([self.user_to_item_etype])
        n_users = full_graph.number_of_nodes(self.user_ntype)
        latest_interactions = dgl.sampling.select_topk(graph_slice, 1, self.timestamp, edge_dir='out')
        user, latest_items = latest_interactions.all_edges(form='uv', order='srcdst')
        # each user should have at least one "latest" interaction
        assert torch.equal(user, torch.arange(n_users))

        recommended_batches = []
        user_batches = torch.arange(n_users).split(self.batch_size)
        for user_batch in user_batches:
            latest_item_batch = latest_items[user_batch].to(device=h_item.device)
            dist = h_item[latest_item_batch] @ h_item.t()
            # exclude items that are already interacted
            for i, u in enumerate(user_batch.tolist()):
                interacted_items = full_graph.successors(u, etype=self.user_to_item_etype)
                dist[i, interacted_items] = -np.inf
            recommended_batches.append(dist.topk(K, 1)[1])

        recommendations = torch.cat(recommended_batches, 0)
        return recommendations


def evaluate_nn(dataset, h_item, k, batch_size):
    g = dataset['train-graph']
    val_matrix = dataset['val-matrix'].tocsr()
    test_matrix = dataset['test-matrix'].tocsr()
    item_texts = dataset['item-texts']
    user_ntype = dataset['user-type']
    item_ntype = dataset['item-type']
    user_to_item_etype = dataset['user-to-item-type']
#     timestamp = dataset['timestamp-edge-column']

    rec_engine = LatestNNRecommender(
        user_ntype, item_ntype, user_to_item_etype, timestamp, batch_size)

    recommendations = rec_engine.recommend(g, k, None, h_item).cpu().numpy()
    return prec(recommendations, val_matrix)

## Run

In [8]:
directory = '/home/mila/r/rebecca.salganik/'
with open(directory + 'dataset_without_im.pkl', 'rb') as f:
        dataset = pickle.load(f)

In [9]:
g = dataset['train-graph']
val_matrix = dataset['val-matrix'].tocsr()
test_matrix = dataset['test-matrix'].tocsr()
item_texts = dataset['item-texts']
user_ntype = dataset['user-type']
item_ntype = dataset['item-type']
user_to_item_etype = dataset['user-to-item-type']

In [10]:
device = torch.device('cuda:0')

In [11]:
# Assign user and movie IDs and use them as features (to learn an individual trainable
# embedding for each entity)
g.nodes['playlist'].data['id'] = torch.arange(g.number_of_nodes('playlist'))
g.nodes['track'].data['id'] = torch.arange(g.number_of_nodes('track'))

In [41]:
# Sampler
batch_sampler = ItemToItemBatchSampler(g, 'playlist', 'track', 32)
neighbor_sampler = NeighborSampler(g, 'playlist', 'track', 
                                   random_walk_length=2, random_walk_restart_prob=0.5, num_random_walks=10, num_neighbors=3, num_layers=2)
collator = PinSAGECollator(neighbor_sampler, g, 'track')

dataloader = DataLoader(
    batch_sampler,
    collate_fn=collator.collate_train,
    num_workers=4) 

dataloader_test = DataLoader(
    batch_sampler,
    collate_fn=collator.collate_test,
    num_workers=4) 

In [27]:
dataloader_test_it = iter(dataloader_test)

In [44]:
for block in iter(dataloader_test_it): 
    print(block)
    break

TypeError: Caught TypeError in DataLoader worker process 1.
Original Traceback (most recent call last):
  File "/home/mila/r/rebecca.salganik/.conda/envs/nsv3/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/mila/r/rebecca.salganik/.conda/envs/nsv3/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 40, in fetch
    return self.collate_fn(data)
  File "/tmp/ipykernel_98198/2478922114.py", line 88, in collate_test
    batch = torch.LongTensor(samples)
TypeError: only integer tensors of a single element can be converted to an index


In [18]:
#Model
model = PinSAGEModel(g, 'track', 128, 2)
model = model.cuda()

In [14]:
# Optimizer
opt = torch.optim.Adam(model.parameters(), lr=3e-5)

In [17]:
dataloader_it = iter(dataloader)

In [21]:
num_epochs = 1 
batches_per_epoch = 100 
losses = []
batch_size = 32 
# For each batch of head-tail-negative triplets...
for epoch_id in range(num_epochs):
    model.train()
    for batch_id in tqdm.trange(batches_per_epoch):
        pos_graph, neg_graph, blocks = next(dataloader_it)
        # Copy to GPU
        for i in range(len(blocks)):
            blocks[i] = blocks[i].to(device)
        pos_graph = pos_graph.to(device)
        neg_graph = neg_graph.to(device)

        loss, auc = model(pos_graph, neg_graph, blocks)
        loss = loss.mean() 
        opt.zero_grad()
        loss.backward()
        opt.step()
    if epoch_id % 10 == 0:
        print(loss, auc)
        losses.append([loss.item(),auc])

    # Evaluate
    model.eval()
    with torch.no_grad():
        item_batches = torch.arange(g.number_of_nodes(item_ntype)).split(batch_size)
        h_item_batches = []
        for blocks in dataloader_test:
            for i in range(len(blocks)):
                blocks[i] = blocks[i].to(device)

            h_item_batches.append(model.get_repr(blocks))
        h_item = torch.cat(h_item_batches, 0)

        print(evaluation.evaluate_nn(dataset, h_item, k, batch_size))

100%|██████████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 12.18it/s]


tensor(1.6530, device='cuda:0', grad_fn=<MeanBackward0>) 0.7166666666666668


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/mila/r/rebecca.salganik/.conda/envs/nsv3/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/mila/r/rebecca.salganik/.conda/envs/nsv3/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 40, in fetch
    return self.collate_fn(data)
  File "/tmp/ipykernel_98198/2478922114.py", line 88, in collate_test
    batch = torch.LongTensor(samples)
TypeError: only integer tensors of a single element can be converted to an index
